# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup#
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Latitude,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saldanha,-33.0117,17.9442,19.03,57,94,4.37,ZA,1745008253
1,1,borogontsy,62.6683,131.1683,-7.96,96,86,1.52,RU,1745008254
2,2,albany,42.6001,-73.9662,20.36,30,100,1.79,US,1745008255
3,3,waitangi,-43.9535,-176.5597,16.01,94,99,3.13,NZ,1745008256
4,4,grytviken,-54.2811,-36.5092,2.35,76,70,2.97,GS,1745008257


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Create a map plot for the city_data_df DataFrame, enabling a geogrpahic plot with color based on city names
city_map = city_data_df.hvplot.points(
    "Lng",                     
    "Latitude",                     
    geo=True,                  
    tiles="CartoLight",        
    size="Humidity",           
    color="City",              
    alpha=0.8,                 
    frame_width=700,           
    frame_height=500           
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"] > 21) & 
    (city_data_df["Max Temp"] < 27) & 
    (city_data_df["Wind Speed"] < 4.5) & 
    (city_data_df["Cloudiness"] == 0)
].copy()


# Drop any rows with null values and display info about remaining cities
ideal_cities_df = ideal_cities_df.dropna()
print(f"Found {len(ideal_cities_df)} cities with ideal weather conditions")
ideal_cities_df

Found 9 cities with ideal weather conditions


,City_ID,City,Latitude,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
178,178,tamanrasset,22.7850,5.5228,25.95,9,0,2.57,DZ,1745008462
198,198,karratha,-20.7377,116.8463,25.97,33,0,2.13,AU,1745008485
224,224,pisco,-13.7000,-76.2167,23.03,64,0,3.60,PE,1745008514
248,248,hadibu,12.6500,54.0333,26.58,64,0,0.76,YE,1745008543
317,317,grand baie,-20.0182,57.5802,24.14,87,0,0.45,MU,1745008629
384,384,cidreira,-30.1811,-50.2056,25.29,78,0,1.97,BR,1745008709
394,394,al ghayzah,16.2079,52.1760,26.23,71,0,2.45,YE,1745008723
504,504,muri,23.3667,85.8667,26.32,68,0,2.07,IN,1745008853
510,510,iranshahr,27.2025,60.6848,26.19,15,0,4.12,IR,1745008860


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Latitude", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df


,City,Country,Latitude,Lng,Humidity,Hotel Name
178,tamanrasset,DZ,22.7850,5.5228,9,
198,karratha,AU,-20.7377,116.8463,33,
224,pisco,PE,-13.7000,-76.2167,64,
248,hadibu,YE,12.6500,54.0333,64,
317,grand baie,MU,-20.0182,57.5802,87,
384,cidreira,BR,-30.1811,-50.2056,78,
394,al ghayzah,YE,16.2079,52.1760,71,
504,muri,IN,23.3667,85.8667,68,
510,iranshahr,IR,27.2025,60.6848,15,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:


# Set parameters to search for a hotel
radius = 10000  # 10 km search radius
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Create a new column for storing hotel names
hotel_df["Hotel Name"] = ""

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = row["Latitude"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
tamanrasset - nearest hotel: فندق الأمان
karratha - nearest hotel: Karratha International Hotel
pisco - nearest hotel: La Portada
hadibu - nearest hotel: No hotel found
grand baie - nearest hotel: Safari
cidreira - nearest hotel: Hotel Castelo
al ghayzah - nearest hotel: فندق تاج العرب
muri - nearest hotel: No hotel found
iranshahr - nearest hotel: هتل قصِر


,City,Country,Latitude,Lng,Humidity,Hotel Name
178,tamanrasset,DZ,22.7850,5.5228,9,فندق الأمان
198,karratha,AU,-20.7377,116.8463,33,Karratha International Hotel
224,pisco,PE,-13.7000,-76.2167,64,La Portada
248,hadibu,YE,12.6500,54.0333,64,No hotel found
317,grand baie,MU,-20.0182,57.5802,87,Safari


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    x="Lng",
    y="Latitude",
    geo=True,
    tiles="CartoLight",
    size="Humidity",
    color="City",
    alpha=0.8,
    hover_cols=["City", "Hotel Name", "Humidity"],
    frame_width=700,
    frame_height=500
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Latitude]   (City,Humidity,Hotel Name)